# Pre-Processing

Úlohou predprocesora je vytvorenie simulačného modelu, prostredia a vygenerovanie vstupných dát pre simulačné prostredie. Vytvorený model sa často používa po simulácii v post-procesore pri zobrazení výsledkov, pretože výsledkom simulácie bývajú zvyčajne dáta o simulovanom jave (napr. intenzita elektrického poľa) a vlastný model nemusí byť súčasťou týchto dát.      

## Vytvorenie modelu pomocou programu *gmsh*

[Gmsh](http://gmsh.info/) je 2D/3D generátor siete modelu, je možno ho použiť aj na zobrazenie výsledkov simulácií. Je súčasťou komplexného prostredia [OneLab](http://onelab.info/), v ktorom je integrovaný univerzálny solver pre riešenie problémov. AKtuálne je podľa štatistík (Elmer) najpoužívanejším generátorom v akademickom prostredí (44%).

Vytvorenie modelu pozostáva z definovania elementárnych entít - bodov, spojovacích línií, vytvorení plôch a objemov. Na základe elementárnych entít sú definované fyzické skupiny, z ktorých sú potom generované siete pre konečné prvky. Fyzické skupiny môžu pozostávať z viacerých elementárnych entít.

Pre vytvorenie modelu môžeme použiť niekoľko postupov, prípadne ich vzájomne kombinovať.

### Vytvorenie modelu manuálne v GUI

Manuálnym zadaním je vhodné vytvárať len veľmi jednoduché modely pre overenie funkčnosti prostredia (aj keď tvorcovia *gmsh* sú iného názoru), konfigurácií a v pedagogickom procese pri demonštrácii princípov pri vytváraní modelu. 

In [1]:
import os
_ = os.system("rm *.geo")
_ = os.system("gmsh")

**Postup**

* Vytvorenie sústavy bodov
  * *Modules -> Geometry -> Elementary entities -> Add -> Point*
* Vytvorenie prepojovacích línií - sú orientované (začiatok - koniec)  
  * *Modules -> Geometry -> Elementary entities -> Add -> Line*
* Vytvorenie plôch - 2D plocha
  * *Modules -> Geometry -> Elementary entities -> Add -> Plane surface*
  * Referenčným bodom plochy je jej stred
  
  
* Konverzia elementárnej plochy na fyzický komponent (môže pozostávať napr. z viacerých logických plôch a pod.)
  * *Modules -> Geometry -> Physical groups -> Add -> Surface*
  
* Vytvorenie siete
  * *Mesh -> Define - 2D*
* Zjemnenie siete
  * *Mesh -> Define - Refine by splitting*
  
Výsledok vytvárania modelu a siete sa automaticky zapisuje do súboru *\*.geo*, ktorý je možné editovať a použiť ako skript (nižšie).

**Príklad vygenerovanej siete**

<img src="./img/gmsh_01.png" width=450 >

### Vytvorenie modelu skriptom

*Gmsh* má vlastný skriptovací jazyk pre definovanie modelov, jednotlivé príkazy sa zadávajú do súboru, ktorý sa načíta pri spustení programu. Pri zadaní bodov zárovň definujeme aj hustotu generovanej siete v okolí bodu.

**Príklad fragmentu skriptu zo súboru ex_01.geo** 

    // elementarne entity                                                              
    Point(1) = {-10,  10, 0, 1.0};                                // bod x,y,z,hustota siete v okoli bodu
    Point(4) = { 10,  10, 0, 1.0};
    Line(1) = {1, 4};                                             // usecka z dvoch bodov
    Line Loop(5) = {3, 4, 1, 2, 13, 14, 11, 12, 23, 24, 21, 22};  // vytvorenie obrysu - uzatvorenej krivky
    Plane Surface(6) = {5};                                       // vytvorenie plochy z obrysu    
    ...
    
    // fyzicke entity
    Physical Line("e1") = {11, 14, 13, 12};                       // lomena ciara zlozena z useciek
    ..
    Physical Surface("air") = {6};                                // fyzicka plocha - moze byt zlozena z viacerych 
    
**Model vygenerovaný skriptom ex_01.geo**

<img src="./img/gmsh_02.png" width=450 >

In [2]:
_ = os.system("gmsh ./data/ex_01.geo")

### Vytvorenie parametrického modelu  skriptom

Pre problémy spojené so zmenou parametrov modelu, napríklad pre potreby optimalizácie, je potrebné mať model alebo jeho časi zadané pomocou parametra, ktorý môžeme meniť. Pre generovanie parametrického modelu existujú podporné programy a knižnice, jedným z nich je knižnica [pygmsh](https://pypi.org/project/pygmsh/). 

V **pygmsh** je možné model vytvoriť pomocou API

  * emulujúcich príkazy skriptu *gmsh* (add_point, add_line ...)
  * funkcií kompatibilných s [openCASCADE](https://dev.opencascade.org/doc/overview/html/technical_overview.html) (add_ball, add_box ...), tieto sú primárne určené pre modelovanie 3D objektov

In [3]:
# Priklad vytvorenie modelu pomocou openCASCADE
# Vytvorenie 2D struktury a extrudovanie do 3D modelu

import pygmsh

geom = pygmsh.opencascade.Geometry(
  characteristic_length_min=0.1,
  characteristic_length_max=0.1,
  )

# Vytvorenie 2D objektov na ploche
rectangle = geom.add_rectangle([-1.0, -1.0, 0.0], 2.0, 2.0)
disk1 = geom.add_disk([-1.2, 0.0, 0.0], 0.5)
disk2 = geom.add_disk([+1.2, 0.0, 0.0], 0.5)
union = geom.boolean_union([rectangle, disk1, disk2])

disk3 = geom.add_disk([0.0, -0.9, 0.0], 0.5)
disk4 = geom.add_disk([0.0, +0.9, 0.0], 0.5, char_length=0.1)
flat = geom.boolean_difference([union], [disk3, disk4])

# Vytvorenie 3D objektu
geom.extrude(flat, [0, 0, 0.3])

# zapis vygenerovaneho objektu v gmsh formate 
fd = open('./data/ex_02.geo', 'w')
fd.write(geom.get_code())
fd.close()

# vygenerovanie siete a zapis do suboru *.msh 
points, cells, point_data, cell_data, field_data = pygmsh.generate_mesh(geom, verbose=False)

In [4]:
# zobrazenie vygenerovaneho *.geo suboru
_ = os.system("gmsh ./data/ex_02.geo")

In [5]:
# zobrazenie vygenerovanej siete
# !!! mriezka neobsahuje fyzicke entity 
_ = os.system("gmsh ./data/ex_02.msh")

**Model vygenerovaný skriptom**

<img src="./img/gmsh_03.png" width=450 >

V *gmsh* môžeme vytvoriť sieť a objekt exportovať do formátu [*VTK*](http://www.cacr.caltech.edu/~slombey/asci/vtk/vtk_formats.simple.html) (*Visualisation Toolkit*), ktorý vie načítať program pre pos-processing [*ParaView*](https://www.paraview.org/).

In [6]:
_ = os.system("paraview ./data/ex_02.vtk")

**Zobrazenie modelu v ParaView**

<img src="./img/para_01.png" width=450 >

## Tvorba modelu pomocou CAD programov

Komplexnejšie modely je možné tvoriť pomocou open-source CAD programov, FreeCAD poskytuje aj možnosť skriptovania modelu v Pythone. Typ modelu závisí od predpokladaného typu simulácie, napr. pre simuláciu elektrostatického poľa - kde nás zaujíma tvar elektrického poľa v prostredí - vytvárame model v tvare dutín v prostredí, kde jednotlivé plochy dutín reprezentujú elekródy so stanoveným potenciálom alebo okrajové podmienky.

**Vytvorenie modelu vo FreeCAD**

<img src="./img/fc_01.png" width=450 >

In [7]:
_ = os.system("freecad ./data/ex_03.fcstd")

Vo FreeCad-e exportujeme model vo formáte BREP, ktorý vie čítať generátor mriežky *gmsh*. 

In [8]:
_ = os.system("gmsh ./data/ex_03.brep")

Súbor *\*.brep* obsahuje model rozložený na elementárne entity, neobsahuje ale fyzické entity. Mená objektov z FreeCAD-u nie sú súčasťou formátu *\*.brep* (a zrejme by to prinášalo problémy), preto musíme jednotlivé fyzické entity "poskladať" ručne - doplnením do súboru *\*.geo*. 

Zásadným rozdielom medzi elementárnou a fyzickou entitou je tem, že jedna fyzická entita (napr. elektróda, zemná plocha atď.) môže byž poskladaná z viacerých fyzických entít. 

Naopak - jeden objekt v CAD-e (napr. valec) môže (ako sústava 3 entít - dno, dekel, plášť) reprezentovať samostatné fyzické entity (2 elektródy spojené izolátorom).

**Súbor ex_03.geo**    

Do súboru importujeme vygenerovaný súbor *ex_03.brep* a doplníme definície fyzických entít. Nepríjemné je hlavne určenie jednotlivých plôch objektov, naštastie sú generované v jednotlivých objektoch v poradí a pomôcť si môžeme aj zobrazením označenia entít v *gmsh*. 
    
    Merge "exp_03.brep";
    Physical Volume("air") = {1};

    Physical Surface("s2")  = {7,8,9};
    Physical Surface("q1")  = {10,11,12};
    Physical Surface("ext") = {1,2,3,4,5,6};

In [9]:
_ = os.system("gmsh ./data/ex_03.geo")

## Generovanie siete

V program *gmsh* môžeme vytvárať mriežku podľa potreby jej zjemnením, vyhladením a pod. Program (ver. 4.0 a vyššia) automaticky pri ukladaní zvolí formát mriežky vo verzii 4, ktorú simulačný engine ElmerCSC (zatiaľ) nepozná, preto je potrebné pri ukladaní zvoliť export mriežky vo formáte 2.0 (ASCII 2).

Vygenerovanú sieť môžeme skontrolovať v grafickom rozhraní ElmerGUI (s obmedzeniami ...).

<img src="./img/elmer_01.png" width=450 >

## Generovanie siete skriptom

Pri častejších zmenách modelu je ručné opakované generovanie mriežky nepríjemné, môžeme si pomôcť vygenerovaním siete priamo v súbore *\*.geo* 

**Súbor ex_04.geo** 

    Merge "ex_03.brep";
    Physical Volume("air") = {1};

    Physical Surface("s2")  = {7,8,9};
    Physical Surface("q1")  = {10,11,12};
    Physical Surface("ext") = {1,2,3,4,5,6};

    Mesh 3;
    RefineMesh;
    RefineMesh;
    
sieť vygenerujeme a zapíšeme v správnom formáte v povelovom riadku príkazom

     gmsh ex_04.geo -3 -format msh2 -save_all -o ex_04.msh 